In [ ]:
# Import warnings
import warnings
warnings.filterwarnings('ignore')

# Import data manipulation libraries
import pandas as pd
import numpy as np
from pprint import pprint
from collections import OrderedDict
from numpy.linalg import multi_dot
from scipy import stats
from tabulate import tabulate

# Import plotly express
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600
# Set precision
pd.set_option('display.precision', 4)

In [ ]:
# Import & Initialize database
import sqlalchemy
engine = sqlalchemy.create_engine('sqlite:///India')

In [ ]:
# Specify assets / stocks
assets = sorted(['ICICIBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT'])
print(assets)

In [ ]:
# Query close price from database
# Refer lab 3 and 1 for further details
df = pd.DataFrame()

for asset in assets:
    df1 = pd.read_sql_query(f'SELECT Date, Close FROM {asset}', engine, index_col='Date')
    df1.columns = [asset]
    df = pd.concat([df, df1], axis=1)

# View dataframe
df

In [ ]:
# Calculate daily returns
returns = df.pct_change().dropna()
returns.head()

In [ ]:
# Stock returns
stockreturn = returns['ICICIBANK']
# Calculate mean and standard deviation
mean = np.mean(stockreturn)
stdev = np.std(stockreturn)
# Calculate VaR at difference confidence level
VaR_90 = stats.norm.ppf(1-0.90,mean,stdev)
VaR_95 = stats.norm.ppf(1-0.95,mean,stdev)
VaR_99 = stats.norm.ppf(1-0.99,mean,stdev)

In [ ]:
# number of stdev from the mean
stats.norm.ppf(0.01)

In [ ]:
# Ouput results in tabular format
table = [['90%', VaR_90],['95%', VaR_95],['99%', VaR_99] ]
header = ['Confidence Level', 'Value At Risk']
print(tabulate(table,headers=header))

In [ ]:
# Use quantile function for Historical VaR
hVaR_90 = returns['ICICIBANK'].quantile(0.10)
hVaR_95 = returns['ICICIBANK'].quantile(0.05)
hVaR_99 = returns['ICICIBANK'].quantile(0.01)

In [ ]:
# Ouput results in tabular format
htable = [['90%', hVaR_90],['95%', hVaR_95],['99%', hVaR_99]]
print(tabulate(htable,headers=header))

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

# Number of simulations
n_sims = 5000

# Simulate returns and sort
sim_returns = np.random.normal(mean, stdev, n_sims)
# Use percentile function for MCVaR
MCVaR_90 = np.percentile(sim_returns,10)
MCVaR_95 = np.percentile(sim_returns, 5)
MCVaR_99 = np.percentile(sim_returns,1)

In [ ]:
# Ouput results in tabular format
mctable = [['90%', MCVaR_90],['95%', MCVaR_95],['99%', MCVaR_99]]
print(tabulate(mctable,headers=header))

In [ ]:
# normality test
stats.shapiro(stockreturn)

In [ ]:
# normality test
stats.anderson(stockreturn)

In [ ]:
# Plot histogram
px.histogram(returns,
    histnorm='probability density',
    title='Histogram of Returns',
    barmode='relative')

In [ ]:
# First four moments
dist = OrderedDict({
    'Mean': np.mean(returns['ICICIBANK']),
    'Variance': np.std(returns['ICICIBANK']),
    'Skew': stats.skew(returns['ICICIBANK']),
    'Kurtosis': stats.kurtosis(returns['ICICIBANK'])
    })

pprint(dist)

In [ ]:
# Specify params for modified VaR
z = abs(stats.norm.ppf(0.01))
s = stats.skew(stockreturn)
k = stats.kurtosis(stockreturn)
t = z+1/6*(z**2-1)*s+1/24*(z**3-3*z)*k-1/36*(2*z**3-5*z)*s**2

# Calculate VaR at difference confidence level
mVaR_99 = (mean-t*stdev)
mVaR_99

In [ ]:
# VaR Scaling
forecast_days = 5
f_VaR_90 = VaR_90*np.sqrt(forecast_days)
f_VaR_95 = VaR_95*np.sqrt(forecast_days)
f_VaR_99 = VaR_99*np.sqrt(forecast_days)

In [ ]:
# Ouput results in tabular format
ftable = [['90%', f_VaR_90],['95%', f_VaR_95],['99%', f_VaR_99] ]
fheader = ['Confidence Level', '5-Day Forecast Value At Risk']
print(tabulate(ftable,headers=fheader))

In [ ]:
# Plot Scaled VaR
sVaR = pd.DataFrame([-100*VaR_99*np.sqrt(x) for x in range(100)], columns=['ScaledVaR'])
px.scatter(sVaR, sVaR.index,'ScaledVaR',title='Scaled VaR', labels={'index':'Horizon'})

In [ ]:
# Calculate CVar
CVaR_90 = returns['ICICIBANK'][returns['ICICIBANK']<=hVaR_90].mean()
CVaR_95 = returns['ICICIBANK'][returns['ICICIBANK']<=hVaR_95].mean()
CVaR_99 = returns['ICICIBANK'][returns['ICICIBANK']<=hVaR_99].mean()

In [ ]:
# Ouput results in tabular format
ctable = [['90%', CVaR_90],['95%', CVaR_95],['99%', CVaR_99] ]
cheader = ['Confidence Level', 'Conditional Value At Risk']
print(tabulate(ctable,headers=cheader))

In [ ]:
# Weights from Minimum Variance Portfolio
wts = np.array([2.553e-01, 4.434e-02, 2.944e-01, 8.664e-02, 3.193e-01])

# Portfolio mean returns and volatility
port_mean = wts.T @ returns.mean()
port_stdev = np.sqrt(multi_dot([wts.T, returns.cov(), wts]))
pVaR = stats.norm.ppf(1-0.99, port_mean, port_stdev)

print(f"Mean: {port_mean}, Stdev: {port_stdev}, pVaR: {pVaR}")

In [ ]:
# Import arch library
from arch import arch_model

In [ ]:
# Mean zero
g1 = arch_model(stockreturn, vol='GARCH', mean='Zero', p=1, q=1, dist='Normal')
model = g1.fit()

In [ ]:
# Model output
print(model)

In [ ]:
model.summary()

In [ ]:
# Model params
model.params

In [ ]:
# Model Confidence Interval
model.conf_int(alpha=.01)

In [ ]:
# Plot annualised vol
fig = model.plot(annualize='D')

In [ ]:
# Forecast for next 5 days
model_forecast = model.forecast(horizon=60)

# Plot forecasted volatility
fdf = pd.DataFrame(np.sqrt(model_forecast.variance.dropna().T *252)*100)
fdf.columns = ['Cond_Vol']
px.scatter(fdf, fdf.index, 'Cond_Vol', labels={'index':'Horizon'}, title='GARCH Volatility Forecast')